In [98]:
import tkinter as tk
import json
import requests
import random
import time

points = 0
questions_json = []
result_text = ""

# Sets the global variable 'questions_json' to the json with the questions/answers set
def generate_json():
    global questions_json
    link = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
    genius = requests.get(link)
    questions_json = genius.json()

# Gets the individual question from the json and returns it
def get_question(q_num):
    question = questions_json[q_num]['question']
    return question

# Compiles a list of the potential answers for a given question, shuffles, and returns them
def get_ans_list(q_num):
    ans_correct = questions_json[q_num]['correctAnswer']
    ans_wrong = questions_json[q_num]['incorrectAnswers']
    ans_list = []
    ans_list.append(ans_correct)
    for i in range(3):
        ans_list.append(ans_wrong[i])
    random.shuffle(ans_list)
    return ans_list

# At a given question index, checks if the selected answer is the correct one
def check_ans(ans, q_num):
    global points
    global questions_json
    
    if ans == questions_json[q_num]['correctAnswer']:
        points += 1
        return("Correct")

    if not ans == questions_json[q_num]['correctAnswer']:
        return("Incorrect")

# Destroys all widgets in the root and generates new ones, ie. Title, Questions, Result text, etc.
def new_question(q_num, choice):
    global questions_json
    global result_text
    
    result = check_ans(choice, q_num)
    result_text.delete(0.0, tk.END)
    result_text.insert(tk.END, result)
    
    del questions_json[q_num]
    
    root.after(2000, destroy)
    
def destroy():
    for widget in root.winfo_children():
        widget.destroy()
    generate_question_ui()

# Generates the Title, Questions, Result text, etc. if 'questions_json' is not empty. If it is, then a blank canvas with the score is displayed
def generate_question_ui():
    start_button.pack_forget()
    global questions_json
    global points
    global result_text
        
    if questions_json:
        q_num = random.randint(0, len(questions_json) - 1)

#         Creates green background
        canvas = tk.Canvas(root, bg= "PaleGreen1", bd= 10, relief= "groove", height= 30, width= 70)
        canvas.pack()

#         Puts the question on the canvas
        question = tk.Label(canvas, text= get_question(q_num), bg= "PaleGreen1", bd= 2, relief= "ridge")
        question.pack()

#         Creates a frame for the Radiobutton answer options
        ans_frame = tk.Frame(canvas, width= 30, height= 30, bg= "PaleGreen1")
        ans_frame.pack()
        
        result_text = tk.Text(canvas, bg= "DarkSeaGreen1", width= 15, height= 1)
        result_text.pack()
        result_text.insert(tk.END, "Result:")
        
#         Button that when clicked checks if your answer was correct and generates a new question
        new_q_button = tk.Button(canvas, text= "Submit", command= lambda: new_question(q_num, r.get()))
        new_q_button.pack(anchor= tk.SW)

        r = tk.StringVar()
        r.set(" ")
        ans_list = get_ans_list(q_num)

        ans = questions_json[q_num]['correctAnswer']

#         Creates the answer options and puts them in their frame
        for i in range(4):
            tk.Radiobutton(ans_frame, text= ans_list[i], bg= "PaleGreen1", variable= r, value= ans_list[i]).pack()
        
    if not questions_json:
        canvas = tk.Canvas(root, bg= "PaleGreen1", bd= 10, relief= "groove", height= 30, width= 70)
        canvas.pack()
        score = tk.Label(canvas, text= f"You scored: {points} points!")
        score.pack()

generate_json()
        
root = tk.Tk()
title = tk.Label(root, text= "Trivia!")
title.pack()

start_button = tk.Button(root, text= "Click To Begin", command= lambda: generate_question_ui())
start_button.pack(expand= True)

root.mainloop()
